# Step 0: Import Dependencies
we need three libraries to build our agent
1. **Numpy** for our Qtable
2. **OpenAI Gym**  for our Taxi Environment
3. **Random** to generate random numbers
<ol>*PS: import sys
sys.path.append('location found above') to import gym lib without facing any problems</ol>

In [5]:
import sys
sys.path.append('c:\python38\lib\site-packages')
import numpy as np
import gym
import random

# Step 1: Create the environment
Using OpenAI Gym to render and environment to train our agent

In [6]:
env = gym.make("Taxi-v3")
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



# Step 2: Create the Q-table and initialize it
<li> create our Q-table, to know how much rows (states) and columns (actions) we need, we need to calculate the action_size and the state_size</li>
<li>OpenAI Gym provides us a way to do that: env.action_space.n and env.observation_space.n</li>

In [7]:
action_size = env.action_space.n
print("Action Size ==>", action_size)
state_size = env.observation_space.n
print("State Size ==>", state_size)

Action Size ==> 6
State Size ==> 500


In [8]:
qtable = np.zeros((state_size, action_size))
print(qtable)

[[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


# Step 3: Create the hyperparameters
specifying hyperparameters.

In [9]:
total_episodes = 50000        # Total episodes
total_test_episodes = 100     # Total test episodes
max_steps = 99                # Max steps per episode

learning_rate = 0.7           # Learning rate
gamma = 0.618                 # Discounting rate

# Exploration parameters
epsilon = 1.0                 # Exploration rate
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.01            # Minimum exploration probability 
decay_rate = 0.01             # Exponential decay rate for exploration prob

# Step 4: The Q learning algorithm
Algorithm implementation

In [10]:
# 2 & 3 are explained in the article
# 2. For life or until learning is stopped
for episode in range(total_episodes):
    # Reset the environment
    state = env.reset()
    step = 0
    done = False
    
    for step in range(max_steps):
        # 3. Choose an action a in the current world state (s)
        ## First we randomize a number
        exp_exp_tradeoff = random.uniform(0,1)
        
        ## If this number > greater than epsilon --> exploitation (taking the biggest Q value for this state)
        if exp_exp_tradeoff > epsilon:
            action = np.argmax(qtable[state,:])
        ## Else doing a random choice --> exploration
        else:
            action = env.action_space.sample()
        
        # Take the action (a) and observe the outcome state(s') and reward (r)
        new_state, reward, done, info = env.step(action)
        
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        qtable[state, action] = qtable[state,action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        
        # Our new state is state
        state = new_state
        
        # If done : finish episode
        if done == True:
            break
    
    episode =+ 1
    
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate*episode)
    
    # Q Learning Algorithm Done here

# Step 5: Use our Q-table to play Taxi
<li>After 50 000 episodes, our Q-table can be used as a "cheatsheet" to play Taxi.</li>
<li>By running this cell we can see our agent playing Taxi.
</li>

In [19]:
env.reset()
rewards = []

for episode in range(total_test_episodes):
    state = env.reset()
    step = 0
    done = False
    total_rewards = 0
    #print("****************************************************")
    #print("EPISODE ", episode)
    
    for step in range(max_steps):
        # UNCOMMENT IT IF YOU WANT TO SEE OUR AGENT PLAYING
        # env.render()
        # Take the action (index) that have the maximum expected future reward given that state
        action = np.argmax(qtable[state,:])
        
        new_state, reward, done, info = env.step(action)
        
        total_rewards += reward
        
        if done:
            rewards.append(total_rewards)
            #print ("Score", total_rewards)
            break
        state = new_state
env.close()
print ("Score over time: " +  str(sum(rewards)/total_test_episodes))

Score over time: 7.96
